# Projeto 2 - Sistema de Recomendação para Aplicativos de Contabilidade

Neste projeto o objetivo é construir um sistema de recomendação com a arquitetura Graph Attention Network – GAT através do mecanismo de atenção em uma rede convolucional de grafos. O projeto é em Linguagem Python com PyTorch e PyTorch Geometric.Ao final do projeto o sistema será capaz de recomendar aplicativos de Contabilidade.

## 1. Carga e Instalação dos Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da linguagem Python usada neste Jupyter Notebook é: ', python_version())

Versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [2]:
# Instalação Pytorch
!pip install -q torch==2.0.0

In [3]:
# Instalação Pytorch Geometric
!pip install -q torch_geometric==2.3.1

In [4]:
# Carga dos Pacotes
# Manipulação de Dados
import pandas as pd
import numpy as np

# Pytorch
import torch
import torch.nn.functional as F

# Pytorch-Geometric
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.utils import to_undirected
from torch_geometric.loader import DataLoader

#Scikit-Learn
import sklearn
from sklearn.model_selection import train_test_split

# Remoção de Warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 2 - Sistema de Recomendação App Contabilidade" --iversions

Author: Projeto 2 - Sistema de Recomendação App Contabilidade

numpy          : 1.21.5
pandas         : 1.4.4
sklearn        : 1.0.2
torch          : 2.0.0
torch_geometric: 2.3.1



## 2. Preparação do Conjunto de Dados

In [6]:
# Dataframe de usuários
users = pd.DataFrame({"user_id": [0, 1, 2, 3]})

# Dataframe de apps
aplicativos = pd.DataFrame({"app_id": [0, 1, 2]})

# Dataframe de avaliações (ratings)
ratings = pd.DataFrame({"user_id": [0, 1, 1, 2, 3],
                        "app_id": [0, 0, 1, 2, 2],
                        "rating": [4, 5, 3, 2, 4]})

In [7]:
# Visualiza dados de usuários
users

,user_id
0,0
1,1
2,2
3,3


In [8]:
# Visualiza dados de aplicativos
aplicativos

,app_id
0,0
1,1
2,2


In [9]:
# Visualia dados de ratings
ratings

,user_id,app_id,rating
0,0,0,4
1,1,0,5
2,1,1,3
3,2,2,2
4,3,2,4


In [10]:
# Converta os IDs dos aplicativos para evitar confusão com os IDs dos usuários
ratings["app_id"] += users.shape[0] - 1

In [11]:
# Visualiza as alterações em ratings
ratings

,user_id,app_id,rating
0,0,3,4
1,1,3,5
2,1,4,3
3,2,5,2
4,3,5,4


## 3. Pré-Processamento dos Dados em Formato de Grafo

In [12]:
# Divide os dados em conjuntos de treinamento e teste
train_ratings, test_ratings = train_test_split(ratings, test_size = 0.2, random_state = 42)

In [13]:
# Prepara os nodes de treino
train_source_nodes = torch.tensor(train_ratings['user_id'].values, dtype = torch.long) #torch.long é para aumentar a precisão do cálculo, igual o float32
train_target_nodes = torch.tensor(train_ratings['app_id'].values, dtype = torch.long)

In [14]:
# Visualiza a fonte dos nodes
train_source_nodes

tensor([3, 1, 0, 2])

In [15]:
# Visualiza o target dos nodes
train_target_nodes

tensor([5, 4, 3, 5])

Os dados separados em treino e teste são tranformados nos nodes, ou nós, para o processamente com os grafos. No exemplo acima, temos os nós dos usuários e os nós dos app, porém ainda não temos nada com o que ligar as duas partes.

In [16]:
# Prepara os edges (arestas) de treino
train_edge_index = torch.stack([train_source_nodes, train_target_nodes], dim = 0)
train_edge_index = to_undirected(train_edge_index)
train_edge_attr = torch.tensor(train_ratings['rating'].values, dtype = torch.float32).view(-1,1)

In [17]:
# Visualiza o index
train_edge_index

tensor([[0, 1, 2, 3, 3, 4, 5, 5],
        [3, 4, 5, 0, 5, 1, 2, 3]])

In [18]:
# Visualiza a aresta
train_edge_attr

tensor([[4.],
        [3.],
        [4.],
        [2.]])

Esta etapa criou as arestas, ou seja, o que liga os nodes entre si. Aqui temos o caso a aresta bidirecional, que siginifica que o modelo irá aprender da fonte para o target e ao contrário também. Por isso no index, temos mais registros que nos nodes de treino, os números são os mesmos dos nodes de treino. Então temos o sentido indo e voltando.

Nesse caso, a avaliação determina qual será a distância da aresta para ligar os nodes. 

In [19]:
# Repete o processo anterior para os dados de teste
test_source_nodes = torch.tensor(test_ratings["user_id"].values, dtype = torch.long)
test_target_nodes = torch.tensor(test_ratings["app_id"].values, dtype = torch.long)
test_edge_index = torch.stack([test_source_nodes, test_target_nodes], dim = 0)
test_edge_index = to_undirected(test_edge_index)
test_edge_attr = torch.tensor(test_ratings["rating"].values, dtype = torch.float32).view(-1, 1)

In [20]:
# Número de nodes
num_nodes = users.shape[0] + aplicativos.shape[0]

# Visualiza o número de nodes
num_nodes

7

In [21]:
# Cria os atributos de aresta do conjunto de treinamento
# Função Data do Torch-Geometric
# Função cria o conjunto de dados com grafo, cria uma matriz
train_data = Data(x = torch.eye(num_nodes, dtype = torch.float32),
                  edge_index = train_edge_index, #Indices das arestas
                  edge_attr = train_edge_attr, # Atributos das arestas
                  y = train_edge_attr) # Avaliação dos usuários, o que o modelo vai prever

In [22]:
# Visualiza o grafo
train_data

Data(x=[7, 7], edge_index=[2, 8], edge_attr=[4, 1], y=[4, 1])

##### Descrição dos Itens no Grafo

Essa representação acima é típica de um objeto Data do PyTorch Geometric, que armazena as informações de um grafo em um formato específico. Vamos entender cada componente dessa representação:

x=[7, 7]: Essa é a matriz de atributos dos nós. O primeiro valor (7) indica o número de nós no grafo, e o segundo valor (7) representa o número de atributos (características) para cada nó. Portanto, a matriz de atributos dos nós terá dimensões 7x7.

edge_index=[2, 8]: Essa é a matriz de índices de arestas, que indica as conexões entre os nós. A matriz tem duas linhas (valor 2), onde cada coluna representa uma aresta. O número 8 indica que existem 8 arestas no grafo.

edge_attr=[4, 1]: Essa é a matriz de atributos das arestas. O primeiro valor (4) indica o número de arestas no grafo, e o segundo valor (1) representa o número de atributos (características) para cada aresta. 

y=[4, 1]: Essa é a matriz de rótulos (labels) dos nós ou arestas, geralmente usada como a variável alvo (target) em problemas de aprendizado supervisionado. O primeiro valor (4) indica o número de rótulos, e o segundo valor (1) indica que cada rótulo é unidimensional (escalar). 

In [23]:
# Cria os atributos de aresta do conjunto de teste
test_data = Data(x = torch.eye(num_nodes, dtype = torch.float32), 
                 edge_index = test_edge_index, 
                 edge_attr = test_edge_attr, 
                 y = test_edge_attr)

In [24]:
# Cria os dataloaders (o que é requerido pelo PyTorch)
train_data_loader = DataLoader([train_data], batch_size = 1)
test_data_loader = DataLoader([test_data], batch_size = 1)

## 4. Construção do Modelo

##### Informações Complementares

A célula abaixo tem a arquitetura do modelo, sendo que após a criação da arquitetura ela é executada em diferentes momentos.

    1) A classe init é criada no momento da inicialização do modelo, ou seja, na criação da instância logo em seguida
    2) O método foward é executado durante o treinamento do modelo

***Outras informações***

- Self é referência a própria classe, no caso RecommenderGAT. Em resumo Self é o nome dado a classe que estamos criando, caso seja necessário outra classe, usamos outro nome para diferenciar.
- As camadas conv1 e conv2 fazem o treinamento em si, a camada linear entrega o resultado final

In [25]:
# Classe do modelo
class RecommenderGAT(torch.nn.Module):
    
    # Método construtor
    def __init__(self, num_nodes, hidden_dim = 32, output_dim = 1, heads = 2):
        
        super(RecommenderGAT, self).__init__()
        
        # Primeira camada convolucional de atenção do grafo (GATConv)
        self.conv1 = GATConv(num_nodes, hidden_dim, heads = heads)
        
        # Segunda camada convolucional de atenção do grafo (GATConv)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads = heads)
        
        # Camada linear (fully connected) para produzir a saída final
        self.linear = torch.nn.Linear(hidden_dim * heads, output_dim)

    # Método forward
    def forward(self, data):
        
        # Extração dos atributos dos nós e da matriz de índices de arestas do objeto Data
        x, edge_index = data.x, data.edge_index

        # Aplicação da primeira camada GAT (self.conv1) seguida de uma função de ativação 
        x = self.conv1(x, edge_index)
        
        # Exponential Linear Unit (ELU)
        x = F.elu(x)
        
        # Aplicação de dropout para regularização (com probabilidade de 0,5)
        # O modelo aprende demais e isso garante a generalização da solução
        x = F.dropout(x, p = 0.5, training = self.training)

        x = self.conv2(x, edge_index)
        
        x = F.elu(x)

        # Agregação global para criar uma representação geral do grafo
        x = global_mean_pool(x, torch.zeros(1, dtype = torch.long))

        # Camada linear (self.linear) para gerar a saída final 
        x = self.linear(x)

        return x

In [26]:
# Cria instância do modelo
model = RecommenderGAT(num_nodes = num_nodes)

In [27]:
# Camadas do modelo
model.modules

<bound method Module.modules of RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)>

In [28]:
# Função de erro
loss_fn = torch.nn.MSELoss()

In [29]:
# Otimizador
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [30]:
# Número de épocas
num_epochs = 100

## 5. Treino e Avaliação do Modelo

In [31]:
# Coloca o modelo em modo de treino
model.train()

RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [32]:
# Loop de treino
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_data_loader:
        optimizer.zero_grad()
        predictions = model(batch)
        loss = loss_fn(predictions, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {total_loss / len(train_data_loader)}')

Epoch: 10, Loss: 3.312990427017212
Epoch: 20, Loss: 0.698807954788208
Epoch: 30, Loss: 1.3767046928405762
Epoch: 40, Loss: 0.8533971309661865
Epoch: 50, Loss: 0.7118762731552124
Epoch: 60, Loss: 0.863028347492218
Epoch: 70, Loss: 0.8376479744911194
Epoch: 80, Loss: 0.7227445840835571
Epoch: 90, Loss: 0.9047994613647461
Epoch: 100, Loss: 0.6914180517196655


In [33]:
# Coloca o modelo em modo de avaliação
model.eval()

RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [34]:
# Faz previsões no conjunto de teste
with torch.no_grad():
    test_predictions = model(test_data).numpy()

In [35]:
# Calcula o RMSE comparando as previsões com as avaliações reais
rmse = np.sqrt(np.mean((test_predictions.flatten() - test_data.y.numpy().flatten()) ** 2))
print(f'RMSE no conjunto de teste: {rmse}')

RMSE no conjunto de teste: 1.7575793266296387


## 6. Teste do Sistema de Recomendação

In [36]:
# Faz a previsão com base nos dados de teste
with torch.no_grad():
    previsao = model(test_data).numpy()

In [37]:
# Visualiza a previsão
print(previsao)

[[3.2424207]]


In [38]:
# Cria um DataFrame com as previsões
df_previsoes = pd.DataFrame({
    "user_id": test_ratings["user_id"].values,
    "app_id": test_ratings["app_id"].values,
    "predicted_rating": np.round(previsao.flatten(), 0)
})

In [39]:
# Exibe a previsão
print(df_previsoes)

   user_id  app_id  predicted_rating
0        1       3               3.0


In [40]:
# Itera sobre as linhas do DataFrame
for index, row in df_previsoes.iterrows():
    if row["predicted_rating"] >= 3:
        print(f"Recomendamos o aplicativo {row['app_id']} para o usuário {row['user_id']}.")
    else:
        print(f"Não recomendamos o aplicativo {row['app_id']} para o usuário {row['user_id']}.")

Recomendamos o aplicativo 3.0 para o usuário 1.0.
